In [1]:
import pandas as pd
import numpy as np

In [2]:
rate_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'

# AKAS Cleaning

In [3]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [4]:
#Replace "\N" with np.nan
akas_copy = akas.copy()
#akas.replace({'"\\N"','np.nan'},inplace=True)
akas = akas.replace({'\\N':np.nan})

In [5]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [6]:
#keep only US movies.
akas= akas[(akas['region'] == 'US')]
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [7]:
akas.isna().sum()

titleId                  0
ordering                 0
title                    0
region                   0
language           1400761
types               434634
attributes         1358846
isOriginalTitle       1345
dtype: int64

# Rating Cleaning

In [8]:
rate = pd.read_csv(rate_url, sep='\t', low_memory=False)

In [9]:
rate.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1936
1,tt0000002,5.8,262
2,tt0000003,6.5,1762
3,tt0000004,5.6,177
4,tt0000005,6.2,2571


In [10]:
#check for duplicates
rate.duplicated().sum()

0

In [11]:
#Check for missing values
rate.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [12]:
#Check for unique values
rate.nunique()

tconst           1266510
averageRating         91
numVotes           20980
dtype: int64

In [13]:
#Replace "\N" with np.nan (if any)
rate = rate.replace({'\\N':np.nan})

In [14]:
#Keep only US movies
keepers = rate['tconst'].isin(akas['titleId'])
keepers

0           True
1           True
2          False
3          False
4           True
           ...  
1266505    False
1266506    False
1266507    False
1266508    False
1266509    False
Name: tconst, Length: 1266510, dtype: bool

In [15]:
rate = rate[keepers]
rate.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1936
1,tt0000002,5.8,262
4,tt0000005,6.2,2571
5,tt0000006,5.1,177
6,tt0000007,5.4,808


# Basic Cleaning

In [16]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [17]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [18]:
#check for duplicates
basics.duplicated().sum()

0

In [19]:
#Replace "\N" with np.nan
basics = basics.replace({'\\N':np.nan})

In [20]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [21]:
#Eliminate movies that are null for runtimeMinutes
basics.dropna(subset=['runtimeMinutes'], inplace=True)
basics.dropna(subset=['genres'], inplace=True)
basics.dropna(subset=['startYear'], inplace=True)

In [22]:
basics['runtimeMinutes'].isnull().value_counts()

False    2532440
Name: runtimeMinutes, dtype: int64

In [23]:
basics['runtimeMinutes'].to_frame()

,runtimeMinutes
0,1
1,5
2,4
3,12
4,1
...,...
9519611,49
9519617,43
9519652,11
9519659,27


In [24]:
#Check for missing values
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear               0
endYear           2484733
runtimeMinutes          0
genres                  0
dtype: int64

In [25]:
#Check for unique values
basics.nunique()

tconst            2532440
titleType              10
primaryTitle      1751874
originalTitle     1770061
isAdult                 2
startYear             146
endYear                92
runtimeMinutes        865
genres               2212
dtype: int64

In [26]:
#Check Data types
basics.dtypes

tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult           object
startYear         object
endYear           object
runtimeMinutes    object
genres            object
dtype: object

In [27]:
#keep only titleType==Movie
basics['titleType'].value_counts()

tvEpisode       1184178
short            581219
movie            368540
video            177599
tvMovie           90159
tvSeries          87868
tvSpecial         17440
tvMiniSeries      16594
tvShort            8530
videoGame           313
Name: titleType, dtype: int64

In [28]:
basics = basics[(basics['titleType'] == 'movie')]

In [29]:
#Eliminate movies that include "Documentary" in genre 
basics.drop(basics[basics['genres'].str.contains('Documentary')].index, inplace = True)

In [30]:
#keep startYear 2000-2022
basics['startYear'] = basics['startYear'].astype('int')

In [31]:
basics = basics[(basics['startYear'] >= 2000)]

In [32]:
basics['startYear'].value_counts()

2018    9619
2017    9428
2019    9354
2016    9017
2015    8584
2022    8513
2021    8175
2014    8164
2013    7788
2020    7548
2012    7305
2011    6772
2010    6365
2009    5973
2008    5212
2007    4628
2006    4384
2005    3897
2004    3531
2003    3222
2002    2988
2001    2850
2000    2730
2023     750
2024       3
2025       2
Name: startYear, dtype: int64

In [33]:
keepers = basics['tconst'].isin(akas['titleId'])
keepers

34803       True
61116       True
67669       True
77964      False
86801       True
           ...  
9519334     True
9519343     True
9519382    False
9519427     True
9519511    False
Name: tconst, Length: 146802, dtype: bool

In [34]:
basics = basics[keepers]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


# Info 

In [35]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84987 entries, 34803 to 9519427
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          84987 non-null  object
 1   titleType       84987 non-null  object
 2   primaryTitle    84987 non-null  object
 3   originalTitle   84987 non-null  object
 4   isAdult         84987 non-null  object
 5   startYear       84987 non-null  int32 
 6   endYear         0 non-null      object
 7   runtimeMinutes  84987 non-null  object
 8   genres          84987 non-null  object
dtypes: int32(1), object(8)
memory usage: 6.2+ MB


In [36]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1404528 entries, 5 to 34511180
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1404528 non-null  object
 1   ordering         1404528 non-null  int64 
 2   title            1404528 non-null  object
 3   region           1404528 non-null  object
 4   language         3767 non-null     object
 5   types            969894 non-null   object
 6   attributes       45682 non-null    object
 7   isOriginalTitle  1403183 non-null  object
dtypes: int64(1), object(7)
memory usage: 96.4+ MB


In [37]:
rate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 484937 entries, 0 to 1266496
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         484937 non-null  object 
 1   averageRating  484937 non-null  float64
 2   numVotes       484937 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.8+ MB


# Download

In [38]:
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['title_aka.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [39]:
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [40]:
rate.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [41]:
akas.to_csv("Data/title_aka.csv.gz",compression='gzip',index=False)

# API

In [ ]:
import tmdbsimple as tmdb

In [ ]:
with open('/Users/Michael/.secret/tmdb_api.json') as f:
    login = json.load(f)
login.keys()

In [ ]:
tmdb.API_KEY =  login['api-key']